In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#importing targets and getting just the true false value
dtypes={
    'elapsed_time':np.int32,
    'event_name':'category',
    'name':'category',
    'level':np.uint8,
    'room_coor_x':np.float32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'text':'category',
    'fqid':'category',
    'room_fqid':'category',
    'text_fqid':'category',
    'fullscreen':'category',
    'hq':'category',
    'music':'category',
    'level_group':'category'}

train = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train.csv', dtype=dtypes)
train_labels = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')
test = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/test.csv')

cols = ['session_id', 'index', 'elapsed_time', 'event_name', 'name', 'level', 'page', 
            'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'hover_duration', 
            'text', 'fqid', 'room_fqid', 'text_fqid', 'fullscreen', 'hq', 'music', 'level_group']
categorical_features = ['event_name', 'name','text', 'fqid', 'room_fqid', 'text_fqid','level_group']
numeric_features = [col for col in train.columns if col not in categorical_features]

label_encoder = preprocessing.LabelEncoder()
for feature in categorical_features:
    train[feature] = label_encoder.fit_transform(train[feature])
    test[feature]=label_encoder.fit_transform(test[feature])
train.isna().sum()

to_drop = ['page', 'hover_duration']
to_mean = ['fqid', 'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y', 'text', 'text_fqid']

#hanlde na features(dropped)
for feature in to_drop:
    train.drop(feature, axis=1, inplace=True)
    test.drop(feature, axis=1, inplace=True)

for feature in to_mean:
    t_mean = train[feature].mean()
    st_mean = test[feature].mean()
    train[feature] = train[feature].fillna(t_mean)
    test[feature] = test[feature].fillna(st_mean)

test_session_id = test["session_id"]
test_session_levels = test["session_level"]
test.drop("session_level", axis=1, inplace=True)

y = train_labels['correct']

SGDClf = linear_model.SGDClassifier(max_iter=200, penalty='l2')
SGDClf.fit(train, y)
SGD_preds = SGDClf.predict(test)

SGD_df = pd.DataFrame({'session_id':test["session_id"],
                   'correct': SGD_preds,
                   'session_level': test_session_levels})

import jo_wilder
env = jo_wilder.make_env()
iter_test = env.iter_test()

counter = 0
# The API will deliver two dataframes in this specific order,
# for every session+level grouping (one group per session for each checkpoint)
for (sample_submission, test) in iter_test: 
    ## users make predictions here using the test data
    t= sample_submission
    le = preprocessing.LabelEncoder()
    for feature in categorical_features:
        t[feature]=le.fit_transform(t[feature])
    for feature in to_drop:
        t.drop(feature, axis=1, inplace=True)
    
    #t.drop('session_level', axis=1, inplace=True)
    for feature in to_mean:
        st_mean = t[feature].mean()
        t[feature] = t[feature].fillna(st_mean)
    
    sample_submission['correct'] = SGDClf.predict(t)
    
    ## env.predict appends the session+level sample_submission to the overall
    ## submission
    env.predict(test)
    counter += 1

FileNotFoundError: ignored